In [1]:
import sys
# TO CHANGE
BASEDIR = "../../../.."
sys.path.insert(0, BASEDIR)

In [2]:
import pprint

In [ ]:
from src.kg_model import KnowledgeGraphModel, KnowledgeGraphModelConfig
from src.db_drivers.vector_driver import EmbedderModelConfig
from src.pipelines.memorize import MemPipelineConfig, MemPipeline, LLMUpdatorConfig
from src.pipelines.qa import QAPipeline, QAPipelineConfig

from src.pipelines.qa.query_preprocessing import QueryPreprocessorConfig
from src.pipelines.qa.answers_aggregation import AnswersAggregatorConfig
from src.pipelines.qa.kg_reasoning import KnowledgeGraphReasonerConfig

/home/dzigen/Desktop/Projects/PersonalAI/.pai_venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## Пример работы с QA-пайплайном (по построению графа знаний)

1. Инициализация модели графа знаний

In [4]:
kg_config = KnowledgeGraphModelConfig(
    nodestree_config=None # Модель дерева вершин строиться не будет
)

EMBEDDER_MODEL_PATH = '../../../../models/intfloat/multilingual-e5-small' # PATH TO APPROPRIATE EMBEDDER-MODEL
kg_config.embedders_config['m-e5-small'] = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH)
kg_model = KnowledgeGraphModel(kg_config)

No sentence-transformers model found with name ../../../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


2. Инициализация Memorize-пайплайна

In [5]:
mem_config = MemPipelineConfig(
    updator_config=LLMUpdatorConfig(
        delete_obsolete_info=False # Выключаем механизм по поиску/удалению устревших знаний в графе при добавлении новой информации
    )
)
mem_pipeline = MemPipeline(kg_model, mem_config)

3. Добавление информации в граф знаний

In [6]:
TEXT_EXAMPLES = [
    "Sasha was walking along the highway.", 
    "Masha was walking along the highway.", 
    "The ship was sailing along the water canal.", 
    "The motorboat was sailing along the river."]

In [7]:
extracted_triplets = []
for example in TEXT_EXAMPLES:
    tmp_extracted_triplets, _ = mem_pipeline.remember(example)
    extracted_triplets += tmp_extracted_triplets

In [9]:
kg_model.count_items()

{'graph_info': {'triplets': 31, 'nodes': 19},
 'embeddings_info': {'nodes': 19, 'triplets': 14},
 'nodestree_info': None}

4. Инициализация QA-пайплайна

In [10]:
qa_config = QAPipelineConfig(
    preprocessor_config=QueryPreprocessorConfig( # disabling preprocessing
        denoising_config=None,
        enhancing_config=None,
        decomposition_config=None
    ),
    reasoner_config=KnowledgeGraphReasonerConfig(), # weak-reasoner by default; language will be detected automatically
    aggregator_config=AnswersAggregatorConfig()
)

In [11]:
qa_pipeline = QAPipeline(kg_model, qa_config)

5. QA по графу знаний

In [12]:
answer, rinfo = qa_pipeline.answer("Did Masha walk along the highway?")
print(answer)

Yes


In [13]:
answer, rinfo = qa_pipeline.answer("Did Katya walk along the highway?")
print(answer)

<|NotEnoughtInfo|>


In [5]:
kg_model.clear()